# Algoritmo SAP

### 1. From the subset of papers $V$, which is obtained from WoS, a directed graph $G = \left(V, E\right)$, with all the papers and references, is generated, where each directed edge $\left(i, j\right)$ or $E$ is a citation from paper $p_i$ to $p_j$:
$$p_i \rightarrow p_j$$

#### Importo librerías

In [146]:
import igraph
from itertools import chain
from wostools import CollectionLazy
import numpy
from datetime import datetime

#### Por medio de la librería *wostools*, leemos el archivo ISI que nos devuelve WOS y extraemos los *citation_pairs*

In [147]:
collection = CollectionLazy.from_filenames("savedrecs.txt")
pairs = list(collection.citation_pairs())

#### Creamos la lista de nodos basados en los artículos contenidos en los *pairs*

In [148]:
nodes = list(set(chain.from_iterable(pairs)))

#### Construimos el grafo **dirigido** y creamos los vértices (nodos):

In [149]:
graph = igraph.Graph(directed=True)
graph.add_vertices(nodes)
print(graph)

IGRAPH DN-- 11228 0 --
+ attr: name (v)


#### Ahora creamos los *edges* usando los *citation_pairs*

In [150]:
graph.add_edges(pairs)

In [151]:
print("nodes =", graph.vcount())
print("edges =", graph.ecount())

nodes = 11228
edges = 16754


### 2. Graph $G$ is filtered

### 2.1 The largest connected component is obtained

In [152]:
giant = graph.components(mode="weak").giant()

### 2.2 From the graph obtained in (2.1), loops are eliminated.
### 2.3 From the graph obtained in (2.2), if there are parallel edges with the same address, only one is selected and the remaining ones are eliminated.

In [153]:
giant = giant.simplify()

### 2.4 From the graph obtained in (2.3), vertices with indegree 1 and outdegree 0 are eliminated along with their endges. The filtered graph obbtained in (2.4) is noted by $G^{\prime} = \left(V^{\prime}, E^{\prime}\right)$

In [154]:
valid_nodes = giant.vs.select(
    lambda node: not(node.indegree() == 1 and node.outdegree() == 0)
).indices
giant = giant.subgraph(valid_nodes).simplify()

In [155]:
print("nodes =", giant.vcount())
print("edges =", giant.ecount())

nodes = 2410
edges = 7936


## 3. Construction of set $V_{\text{root}}$

### 3.1 The edges of $V^{\prime}$ with outdegree 0 are selected.

In [156]:
zero_outdegree = giant.vs.select(lambda node: node.outdegree() == 0).indices

### 3.2 The vertices obtained in (3.1) are organized in descending order by their degree of entry
### 3.3 From the vertices ordered in (3.2), the first 10 are selected

In [157]:
nodes_highest_indegree = sorted(giant.vs[zero_outdegree], key=lambda node: node.indegree(), reverse=True)[:10]
highest_indegree = [node.index for node in nodes_highest_indegree]

In [158]:
print(giant.vs[highest_indegree].indegree())

[46, 42, 41, 33, 33, 30, 30, 20, 20, 20]


### 3.4 $V_{\text{root}}$ is defined as the set of all the vertices selected in (3.3). Those vertices are called **roots**

#### Creamos una property *kind* que va almacenar el tipo de los nodos

In [159]:
giant.vs["kind"] = None
giant.vs[highest_indegree]["kind"] = "root"
roots = nodes_highest_indegree

### 3.5 If r is a root, the sap of the r is defined as its indegree.

In [160]:
sap_root = giant.vs[highest_indegree].indegree()
giant.vs["sap"] = None
giant.vs[highest_indegree]["sap"] = sap_root

In [161]:
list(giant.vs[highest_indegree])[:2]

[igraph.Vertex(<igraph.Graph object at 0x7fead8f715e0>, 1751, {'name': 'Skumryev V, 2003, NATURE, V423, P850, DOI 10.1038/nature01687', 'kind': 'root', 'sap': 46}),
 igraph.Vertex(<igraph.Graph object at 0x7fead8f715e0>, 1511, {'name': 'Ferrando R, 2008, CHEM REV, V108, P845, DOI 10.1021/cr040090g', 'kind': 'root', 'sap': 42})]

## 4. Construction of set $V_{\text{leaves}}$

### 4.1 $V^{\prime}$ vertices with indegree 0 are selected

In [162]:
zero_indegree = giant.vs.select(lambda node: node.indegree() == 0).indices
nodes_zero_indegree = giant.vs[zero_indegree]

### 4.2 From the vertices in (4.1), those for which there are at least three paths between this and the set of roots are selected

In [163]:
at_least_three_paths = []
for index in zero_indegree:
    paths = giant.get_all_simple_paths(index, roots)
    num_paths = len(paths)
    if num_paths >= 3:
        at_least_three_paths.append(index)
nodes_at_least_three_paths = giant.vs[at_least_three_paths]

In [164]:
print("zero indegree nodes =", len(zero_indegree))
print("zero indegree nodes with at least 3 paths with roots =",
      len(at_least_three_paths)
)

zero indegree nodes = 146
zero indegree nodes with at least 3 paths with roots = 73


### 4.3 From the vertices in (4.2), those whose age (time sinze publication) do not exceed 5 years from the current year are selected

In [165]:
def extract_year(label):
    return int(label.split(", ")[1])

years = [extract_year(node["name"]) for node in giant.vs]
current_year = datetime.now().year
ages = current_year - numpy.array(years)

giant.vs["age"] = ages

### 4.4 $V_{\text{leaf1}}$ is defines as the set of all vertices selected in (4.3)

In [166]:
leaf_1 = giant.vs.select(
    lambda node: node["age"] <= 5 and node in nodes_at_least_three_paths
)
leaf_1["kind"] = "leaf_1"

In [167]:
print(len(leaf_1))

54


### 4.5 If h belogs to $V_{\text{leaf1}}$, its SAP is denoted as the number of paths that exists between h and the roots

In [168]:
num_paths = []
for node in leaf_1:
    paths = giant.get_all_simple_paths(node, roots)
    num_paths.append(len(paths))

In [169]:
leaf_1["sap"] = num_paths

In [170]:
leaf_1[0]

igraph.Vertex(<igraph.Graph object at 0x7fead8f715e0>, 109, {'name': 'Lu XZ, 2016, CHINESE PHYS B, V25, DOI 10.1088/1674-1056/25/5/053601', 'kind': 'leaf_1', 'sap': 226, 'age': 4})

### 4.6 The vertices of the set $V^{\prime} - \left(V_{\text{leaf1}} \cup V_{\text{root}}\right)$ that are in the paths between the elements of $V_{\text{root}}$ and $V_{\text{leaf1}}$ with which it has a connection by one or more paths

In [171]:
nodes_subset = set(giant.vs) - (set(leaf_1) | set(roots))

In [172]:
len(nodes_subset)

2346

In [173]:
between_leaf1_and_root = set()
for node in leaf_1:
    paths = giant.get_all_simple_paths(node, roots)
    between_leaf1_and_root.update(
        set(chain.from_iterable(paths))
    )

In [174]:
nodes_subset = [node
                for node in nodes_subset
                if node.index in between_leaf1_and_root]

In [175]:
len(nodes_subset)

120

### 4.8 From the vertices in (4.6), those whose age does not exceed five years from the current year are selected

In [176]:
nodes_subset = [node
                for node in nodes_subset
                if node["age"] <= 5]

In [179]:
len(nodes_subset)

41

### 4.7 The sap of the vertices obtained in (4.6) are defined as the sum of the saps of the elements of $V_{\text{root}}$ and $V_{\text{leaf1}}$ with it has a connection by one of more paths

In [180]:
# mucha fuerza bruta aquí, por ahora ...
saps_subset = []
for node_subset in nodes_subset:
    to_sum_saps = set()
    for node in leaf_1:
        for root in roots:
            paths = giant.get_all_simple_paths(node, root)
            if node_subset.index in set(chain.from_iterable(paths)):
                to_sum_saps.update([node, root])
    sap = sum([node["sap"] for node in to_sum_saps])
    saps_subset.append(sap)
    print(sap)

268
5171
1306
16979
1196
861
58703
68710
31599
129
14101
2217
55443
8693
2399
19116
6838
136
35670
16333
4883
26580
355
74934
37428
50296
1244
20482
1238
355
145
1306
7835
5661
1292
31095
45378
16979
28399
321
17783


### 4.9 Let $V_{\text{leaf2}}$ be the set of all the vertices selected in (4.8)

In [185]:
leaf_2 = giant.vs[[node.index for node in nodes_subset]]

### 4.10 If $V \in V_{\text{leaf2}}$, its sap is defined by (4.7)

In [190]:
leaf_2["sap"] = saps_subset
leaf_2["kind"] = "leaf_2"

In [191]:
leaf_2[0]

igraph.Vertex(<igraph.Graph object at 0x7fead8f715e0>, 346, {'name': 'Shao GF, 2015, ACTA PHYS SIN-CH ED, V64, DOI 10.7498/aps.64.013602', 'kind': 'leaf_2', 'sap': 268, 'age': 5})

### 4.11 $V_{\text{leaves}}$ is defined as the union between $V_{\text{leaf1}}$ with $V_{\text{leaf2}}$ and so, $V_{\text{leaves}} = V_{\text{leaf1}} \cap V_{\text{leaf2}}$

In [192]:
leaves = giant.vs.select(lambda node: node["kind"] in ["leaf_1", "leaf_2"])

In [193]:
len(leaves)

95